In [1]:
import os
import math
from PIL import Image

def rotate_and_crop(img, angle):
    # 計算原始尺寸
    w, h = img.size
    radians = math.radians(angle)

    # 計算旋轉後圖片需要放大的比例
    sin = abs(math.sin(radians))
    cos = abs(math.cos(radians))
    new_w = w * cos + h * sin
    new_h = w * sin + h * cos

    # 放大比例計算 (填滿黑邊)
    scale = max(new_w / w, new_h / h)

    # 先放大圖片再旋轉
    enlarged_img = img.resize((int(math.ceil(w * scale)), int(math.ceil(h * scale))), Image.BICUBIC)
    rotated_img = enlarged_img.rotate(angle, resample=Image.BICUBIC, expand=False)

    # 旋轉後直接裁剪至原圖尺寸（避免黑邊）
    rotated_w, rotated_h = rotated_img.size
    left = (rotated_w - w) / 2
    top = (rotated_h - h) / 2
    right = left + w
    bottom = top + h

    final_img = rotated_img.crop((left, top, right, bottom))
    return final_img

def process_images(input_folder, output_folder, step=90):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
            filepath = os.path.join(input_folder, filename)
            try:
                img = Image.open(filepath).convert('RGB')
                basename, ext = os.path.splitext(filename)

                # 每 step 度旋轉一次，儲存圖片
                for angle in range(0, 360, step):
                    rotated_img = rotate_and_crop(img, angle)
                    rotated_img.save(os.path.join(output_folder, f"{basename}_rot{angle}{ext}"))

            except Exception as e:
                print(f"處理圖片 {filename} 時發生錯誤：{e}")

# 使用範例
input_folder = r'E:\Codes\CV train\回收系統dataset\Garbage\addmetal'           # 原圖資料夾
output_folder = r'E:\Codes\CV train\回收系統dataset\Garbage\new_metal'  # 輸出資料夾
process_images(input_folder, output_folder)
